### 임포트

In [1]:
import torch
import os
import sys
import numpy as np
import random
import copy

# custom
from util import *
from transformers import GPT2Tokenizer
from AAC_Prefix.AAC_Prefix import * # network
from Train import *

### 기타 값들 설정

In [2]:
# prefix vector 크기 설정
temporal_prefix_size = 15 # 0 or 15
global_prefix_size = 11 # 0 or 11

prefix_size = temporal_prefix_size + global_prefix_size 

# mapping network가 사용할 transformer의 스펙 설정
transformer_num_layers = {"temporal_num_layers" : 4, "global_num_layers" : 4}
prefix_size_dict = {"temporal_prefix_size" : temporal_prefix_size, "global_prefix_size" : global_prefix_size}


data_dir = './Clotho'
MODEL_NAME = 'add_exp_train_clotho_test_audiocaps'

epochs = 60
LR = 5e-5

TEST_BATCH_SIZE = 5
TRAIN_BATCH_SIZE = 55

random_seed=2766
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True
np.random.seed(random_seed)
random.seed(random_seed)  

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda' if USE_CUDA else 'cpu')

### Tokenizer, Dataloader 불러오기

In [3]:
tokenizer_type = 'GPT2'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab_size = None

In [4]:
train_dataloader = CreateDataloader(tokenizer, data_dir, TRAIN_BATCH_SIZE, 'development', prefix_size, is_TrainDataset = True, tokenizer_type = tokenizer_type)
test_dataloader_audiocaps = CreateDataloader(tokenizer, './AudioCaps', TEST_BATCH_SIZE, 'test', prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)

get dataset...: 100%|███████████████████████| 960/960 [00:00<00:00, 1495.88it/s]


### 학습결과 정리하는 폴더 생성하기

In [5]:
directory = "./Train_record/params_" + MODEL_NAME
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print("Error: Failed to create the directory.")


### 모델 초기화

In [6]:
model = get_AAC_Prefix(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'Clotho',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = True, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

# 다른 데이터셋으로 평가하는데 이 때 조건이 Header는 freeze하는 조건이 들어있었다. 그래서 Header freeze 해줌
for param in model.language_header.parameters():
    param.requires_grad = False

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use GPT2 Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
Encoder freezing
GPT2 freezing
header trainable!


### 학습 & 평가

In [7]:
Train(model, LR, train_dataloader, test_dataloader_audiocaps,
    epochs, model_name = MODEL_NAME, beam_search = True, device = device,
    Dataset = 'Clotho', test_dataloader_other_dataset = None)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Eval using dataset...: 100%|██████████████████| 957/957 [18:44<00:00,  1.17s/it]


loading annotations into memory...
0:00:00.006744
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 59528 tokens at 394543.44 tokens per second.
PTBTokenizer tokenized 11257 tokens at 123190.14 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 9331, 'reflen': 8422, 'guess': [9331, 8374, 7417, 6460], 'correct': [4032, 1281, 411, 93]}
ratio: 1.107931607694003
Bleu_1: 0.432
Bleu_2: 0.257
Bleu_3: 0.154
Bleu_4: 0.085
computing METEOR score...
METEOR: 0.141
computing Rouge score...
ROUGE_L: 0.333
computing CIDEr score...
CIDEr: 0.211
computing SPICE score...


Parsing reference captions
Parsing test captions


...

loading annotations into memory...
0:00:00.006645
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 59528 tokens at 421589.26 tokens per second.
PTBTokenizer tokenized 11690 tokens at 123862.71 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 9701, 'reflen': 8896, 'guess': [9701, 8744, 7787, 6830], 'correct': [4218, 1342, 435, 82]}
ratio: 1.0904901079135465
Bleu_1: 0.435
Bleu_2: 0.258
Bleu_3: 0.155
Bleu_4: 0.082
computing METEOR score...
METEOR: 0.142
computing Rouge score...
ROUGE_L: 0.321
computing CIDEr score...
CIDEr: 0.195
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.5 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

SPICE evaluation took: 12.85 s
SPICE: 0.086
computing SPIDEr score...
SPIDEr: 0.141

Training time : 5:10:08
